In [69]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

import pandas as pd
import os
import copy

def pt(d, t):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def bin_print(ts):
    '''
    Print binary tree
    '''
    Tree.fromstring(str(C_Tree.to_binary_right(C_Tree.from_string(ts)))).pretty_print()

tsa = "(NTA (PA a) (NTB (NTC (NTD (PB b) (NTE (PC c) (NTF (NTG (PD d) (PE e)) (PF f)))) (PG g)) (PH h)))"
tsb = "(NTA (PA a) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"

ts1 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts2 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
ts3 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
ts4 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts5 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
ts6 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
ts7 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
ts8 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
ts9 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"

## Encoding playground

==============================================
== TETRATAG
==============================================

w: lR_NT (+1) => (1) push arbol (NT, [w, N])
w: lL_NT (+G) => (1) push arbol (NT, [w, N]); (2) marcar como el ultimo lL
w: rL_NT (-G) => (1) insert el nodo en N; (2) merge hasta el ultimo lL; (3) push arbol (NT, [top(s), N])
w: rR_NT (-1) => (1) insert el nodo en N; (2) push arbol (NT, [top(s), N]); [SI el -1 va justo despues de un +G, cierra el +G]

POSTAGS  = [PA, PB, PC, PD, PE, PF, PG, PH]
PREORDER = ['a', 'NTA', 'b', 'NTD', 'c', 'NTE', 'd', 'NTG', 'e', 'NTF', 'f', 'NTC', 'g', 'NTB', 'h']
b = terminals		l=labels	s=[]


b = [abcdefgh]			l=[lR lL lR lL rR rL rR rR]	s =
# read lR => (NTA [a ε]) into s

b = [bcdefgh]			l=[lL lR lL rR rL rR rR]	s = (NTA [a ε])
# read lL => (NTD [b ε]) into s and mark it as G

b = [cdefgh] 			l=[lR lL rR rL rR rR]		s = (NTA [a ε]) 	(NTD [b ε])
# read lR => (NTE [c ε]) into s

b = [defgh] 			l=[lL rR rL rR rR]		s = (NTA [a ε]) 	(NTD [b ε]) 	(NTE [c ε])
# read lL => (NTG [d ε]) into s and mark it as G

b = [efgh] 			l=[rR rL rR rR]			s = (NTA [a ε]) 	(NTD [b ε]) 	(NTE [c ε]) 	(NTG [d ε])
# read rR => merge + inherit + push (NTF [(NTG [d e]) ε]); as (NTG [d ε]) has already been merged remove as last G

b = [fgh] 			l=[rL rR rR]			s = (NTA [a ε]) 	(NTD [b ε]) 	(NTE [c ε]) 	(NTF [(NTG [d e]) ε])
# read rL => merge until last G + inherit + push (NTC [(NTD [b (NTE [c (NTF  [(NTG [d e]) f])]ε])) g]); as (NTD [b ε]) has already been merged remove as last G

b = [gh] 			l=[rR rR]			s = (NTA [a ε]) 	(NTC [(NTD [b (NTE [c (NTF  [(NTG [d e]) f])]ε])) g])
# read rR => merge + inherit + push (NTB [(NTC (NTD [b (NTE [c (NTF [(NTG [d e]) f])]ε])]) g) ε])

b = [h] 			l=[rR]				s = (NTA [a ε]) 	(NTB [(NTC (NTD [b (NTE [c (NTF [(NTG [d e]) f])]ε])]) g) ε])
# read rR => merge + NOT inherit because LAST NODE + push (NTB [(NTC (NTD [b (NTE [c (NTF [(NTG [d e]) f])]ε])]) g) h])

b = []				l=[]				s = (NTA [a ε]) 	(NTB [(NTC (NTD [b (NTE [c (NTF [(NTG [d e]) f])]ε])]) g) h])
# merge as a last step (NTA [a (NTB [(NTC (NTD [b (NTE [c (NTF [(NTG [d e]) f])]ε])]) g) h])])

==============================================
== RELATIVO
==============================================

a	PA	1*_NTA		+1
b	PB	3*_NTD		+G
c	PC	1*_NTE		+1
d	PD	2*_NTG		+G
e	PE	-1*_NTF		-1
f	PF	-3*_NTC		-G
g	PG	-1*_NTB		-1
h	PH	-1*_NTA		-1


In [79]:
ts = tsb
debug = True
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="*")

pt(debug, ts)    
tree = C_Tree.from_string(ts)
lc_tree = encoder.encode(tree)

new_labels = []
for label in lc_tree.labels:
    new_nc = ""
    if lc_tree.labels.index(label)==0:
        new_nc = "G"
    elif lc_tree.labels.index(label)==len(lc_tree.labels)-1:
        new_nc = "-G"
    elif label.n_commons == 1:
        new_nc = "1"
    elif label.n_commons == -1:
        new_nc = "-1"
    elif label.n_commons < 0:
        new_nc = "-G"
    else:
        new_nc = "G"
    lc = label.last_common if lc_tree.labels.index(label)!=len(lc_tree.labels)-1 else "$$"
    new_labels.append((new_nc, lc))

def operator_1(gap_stack, stack, word, last_common):
    # w: lR_NT (+1) => (1) push arbol (NT, [w, N])
    new_tree = C_Tree(label=last_common, children=[C_Tree(label=word, children=C_Tree(label=word)), C_Tree(label="EMPTY")])
    stack.append(new_tree)
    return gap_stack, stack

def operator_n1(gap_stack, stack, word, last_common):
    # w: rR_NT (-1) => (1) insert el nodo en N; (2) push arbol (NT, [top(s), N]); 
    #               =? [SI el -1 va justo despues de un +G, cierra el +G]
    stack_top = stack.pop()

    if stack_top.children[0].label == "EMPTY":
        stack_top.children[0] = C_Tree(label=word, children=C_Tree(label=word))
    else:
        stack_top.children[1] = C_Tree(label=word, children=C_Tree(label=word))
    
    new_tree = C_Tree(label=last_common, children=[stack_top, C_Tree(label="EMPTY")])
    stack.append(new_tree)

    return gap_stack, stack

def operator_open_gap(gap_stack, stack, word, last_common):
    # w: lL_NT (+G) => (1) push arbol (NT, [w, N]); (2) marcar como el ultimo lL
    new_tree = C_Tree(label=last_common, children=[C_Tree(label=word, children=C_Tree(label=word)), C_Tree(label="EMPTY")])
    stack.append(new_tree)
    gap_stack.append(new_tree)
    return gap_stack, stack

def operator_close_gap(gap_stack, stack, word, last_common):
    # w: rL_NT (-G) => (1) insert el nodo en N; (2) merge hasta el ultimo lL; (3) push arbol (NT, [top(s), N])
    last_open_gap = gap_stack.pop()

    while last_open_gap not in stack:
        last_open_gap = gap_stack.pop()
    last_open_gap_position_in_stack = stack.index(last_open_gap)

    stack_top = stack.pop()
    if stack_top.children[0].label == "EMPTY":
        stack_top.children[0] = C_Tree(label=word, children=C_Tree(label=word))
    else:
        stack_top.children[1] = C_Tree(label=word, children=C_Tree(label=word))
    
    for i in range(last_open_gap_position_in_stack, len(stack)):
        new_top = stack.pop()
        
        # insret on empty
        if new_top.children[0].label == "EMPTY":
            new_top.children[0] = stack_top
        else:
            new_top.children[1] = stack_top
        
        stack_top = new_top
    new_tree = C_Tree(label=last_common, children=[stack_top, C_Tree(label="EMPTY")])
    stack.append(new_tree)

    return gap_stack, stack

gap_stack = []
stack     = []

for label, word in zip(new_labels, lc_tree.words):
    print(label, word)
    if label[0] == "1":
        gap_stack, stack = operator_1(gap_stack, stack, word, label[1])
    elif label[0] == "-1":
        gap_stack, stack = operator_n1(gap_stack, stack, word, label[1])
    elif label[0] == "G":
        gap_stack, stack = operator_open_gap(gap_stack, stack, word, label[1])
    elif label[0] == "-G":
        gap_stack, stack = operator_close_gap(gap_stack, stack, word, label[1])
    else:
        raise ValueError("Unknown label: {}".format(label))

Tree.fromstring(str(stack[0])).pretty_print()


        NTA                                            
  _______|_______________                               
 |                      NTB                            
 |            ___________|_______________               
 |          NTC                         NTG            
 |    _______|___                     ___|___________   
 |   |          NTF                 NTH              | 
 |   |        ___|_______     _______|___            |  
 |   |      NTD          |   |          NTI          | 
 |   |    ___|___        |   |    _______|___        |  
 |   |   |      NTE      |   |   |          NTJ      | 
 |   |   |    ___|___    |   |   |        ___|___    |  
 PA  PB  PC  PD      PE  PF  PG  PH      PI      PJ  PK
 |   |   |   |       |   |   |   |       |       |   |  
 a   b   c   d       e   f   g   h       i       j   k 

('G', 'NTA') a
('G', 'NTC') b
('G', 'NTD') c
('1', 'NTE') d
('-G', 'NTF') e
('-G', 'NTB') f
('G', 'NTH') g
('1', 'NTI') h
('1', 'NTJ') i
('-G', 'NTG') j

In [39]:
all_ts = [tsb]
debug = True
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="*")
for ts in all_ts:
    pt(debug, ts)
    
    tree = C_Tree.from_string(ts)
    lc_tree = encoder.encode(tree)

    if debug:
        print(lc_tree)

    dec_tree = encoder.decode(lc_tree)
    dec_tree = dec_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    pt(debug, dec_tree)
    
    assert str(dec_tree) == ts

        NTA                                            
  _______|_______________                               
 |                      NTB                            
 |            ___________|_______________               
 |          NTC                         NTG            
 |    _______|___                     ___|___________   
 |   |          NTF                 NTH              | 
 |   |        ___|_______     _______|___            |  
 |   |      NTD          |   |          NTI          | 
 |   |    ___|___        |   |    _______|___        |  
 |   |   |      NTE      |   |   |          NTJ      | 
 |   |   |    ___|___    |   |   |        ___|___    |  
 PA  PB  PC  PD      PE  PF  PG  PH      PI      PJ  PK
 |   |   |   |       |   |   |   |       |       |   |  
 a   b   c   d       e   f   g   h       i       j   k 

-BOS-	-BOS-	-BOS-
a	PA	1*_NTA
b	PB	2*_NTC
c	PC	2*_NTD
d	PD	1*_NTE
e	PE	-2*_NTF
f	PF	-2*_NTB
g	PG	2*_NTH
h	PH	1*_NTI
i	PI	1*_NTJ
j	PJ	-3*_NTG
k	PK	-2*_NT

## Labels equivalence

In [2]:
ptb_path = "~/Treebanks/const/PENN_TREEBANK/"
ptb_path = os.path.expanduser(ptb_path)
print("Reading PTB from",ptb_path)    
df = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
with open(os.path.join(ptb_path,"test.trees")) as f:
    ptb = f.read().splitlines()

for tree_string in ptb:
    tree = C_Tree.from_string(tree_string)
    rel_encoder = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=False, binary=True, binary_marker="[*]")
    tet_encoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="[*]")

    t_lin_tree = tet_encoder.encode(copy.deepcopy(tree))
    r_lin_tree = rel_encoder.encode(copy.deepcopy(tree))

    df_temp = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
    
    df_temp["Word"] = r_lin_tree.words
    df_temp["Relative"] = [str(l.n_commons) for l in r_lin_tree.labels]
    df_temp["Tetratag"] = [l.n_commons for l in t_lin_tree.labels]
    
    df = pd.concat([df, df_temp], axis=0)

    
    r_decoded_tree = rel_encoder.decode(r_lin_tree)
    r_decoded_tree = r_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    t_decoded_tree = tet_encoder.decode(t_lin_tree)
    t_decoded_tree = t_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
        
    if not (str(r_decoded_tree)==tree_string):
        print("Decoded tree is different from original")
        print("Original:",tree_string)
        print("Decoded:",r_decoded_tree)

Reading PTB from /home/droca1/Treebanks/const/PENN_TREEBANK/


In [3]:
# print relative values for each of the unique values of tetratag
tetra_values = df["Tetratag"].unique()
tetra_values.sort()
for tetratag in tetra_values:
    print("Tetratag:",tetratag,\
        "\n\t==> Relative:",df[df["Tetratag"]==tetratag]["Relative"].unique())

Tetratag: l 
	==> Relative: ['-3' '-4' '-1' '-2' '-5' '0' '-6' '-7' '-12' '-9' '-14' '-8' '-10' '-11']
Tetratag: lL 
	==> Relative: ['-3' '-4' '-1' '-5' '-13' '-6' '-2' '-11' '-8' '-14' '-7' '-10' '-12'
 '-9' '-19' '-16' '-15' '-17' '-27' '-22' '-20' '-18' '-25' '-21' '-23'
 '-30' '-26']
Tetratag: lR 
	==> Relative: ['-4' '-6' '-5' '-3' '-2' '-13' '-1' '-15' '-7' '-9' '-11' '-8' '-17'
 '-10' '-14' '-12' '-16' '-19']
Tetratag: r 
	==> Relative: ['1']
Tetratag: rL 
	==> Relative: ['1' '2' '0' '3']
Tetratag: rR 
	==> Relative: ['1' '2' '3' '4' '5' '6']
